In [10]:
from sqlalchemy import create_engine

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [11]:
from sqlalchemy import create_engine
database_path = "FPA_FOD_20170508.sqlite"

In [12]:
engine = create_engine(f"sqlite:///{database_path}")
Base.metadata.create_all(engine)

In [13]:
# NOTE: to print the tables of the database
print (engine.table_names())

['ElementaryGeometries', 'Fires', 'KNN', 'NWCG_UnitIDActive_20170109', 'SpatialIndex', 'geometry_columns', 'geometry_columns_auth', 'geometry_columns_field_infos', 'geometry_columns_statistics', 'geometry_columns_time', 'idx_Fires_Shape', 'idx_Fires_Shape_node', 'idx_Fires_Shape_parent', 'idx_Fires_Shape_rowid', 'spatial_ref_sys', 'spatial_ref_sys_aux', 'spatialite_history', 'sql_statements_log', 'sqlite_sequence', 'views_geometry_columns', 'views_geometry_columns_auth', 'views_geometry_columns_field_infos', 'views_geometry_columns_statistics', 'virts_geometry_columns', 'virts_geometry_columns_auth', 'virts_geometry_columns_field_infos', 'virts_geometry_columns_statistics']


In [14]:
from sqlalchemy.orm import Session
session = Session(bind=engine)


# NOTE: use dataframe to see column names
import pandas as pd
fire_df = pd.read_sql('select * from Fires', engine)
fire_df.sample(20)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
565511,565512,609797,SFO-OK01000505-10555_09240921,NONFED,ST-NASF,ST/C&L,USOKOKS,Oklahoma Division of Forestry,OKOK12,OKS District 12,...,C,34.098670,-94.837500,14.0,MISSING/NOT SPECIFIED,OK,McCurtain,089,McCurtain,b'\x00\x01\xad\x10\x00\x00\xd0\xdc\x8e\x99\x99...
351159,351160,360548,W-510952,FED,DOI-WFMI,NPS,USAZGCP,Grand Canyon National Park,AZGCP,Grand Canyon National Park,...,A,36.051017,-112.138642,3.0,NPS,AZ,None,None,None,b'\x00\x01\xad\x10\x00\x00\x84\xf6\xb1\x82\xdf...
934807,934808,1062258,SWRA_TN_14224,NONFED,ST-TNTNS,ST/C&L,USTNTNS,Tennessee Division of Forestry,TNTNS3,TNS Unit 3,...,B,35.656700,-85.346700,14.0,MISSING/NOT SPECIFIED,TN,None,None,None,b'\x00\x01\xad\x10\x00\x00xa2U0VU\xc0H\xa4\xdf...
896711,896712,1022618,SWRA_LA_24857,NONFED,ST-LALAS,ST/C&L,USLALAS,Louisiana Office of Forestry,LALAS4,LAS District 4,...,C,32.855300,-94.000000,14.0,MISSING/NOT SPECIFIED,LA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xfc\xff\xff\xff\xff...
1111416,1111417,1353510,CDF_1996_54_2215_10485,NONFED,ST-CACDF,ST/C&L,USCABTU,Butte Unit,CABTU,Butte Unit,...,A,39.415000,-121.603889,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00$\x81\x8b\x1d\xa6f^\...
522471,522472,562659,SFO-NC0395-00307,NONFED,ST-NASF,ST/C&L,USNCNCS,North Carolina Forest Service,NCNCS,North Carolina Division of Forest Resources,...,B,35.952240,-76.938970,14.0,MISSING/NOT SPECIFIED,NC,Bertie,015,Bertie,b'\x00\x01\xad\x10\x00\x004{\xa0\x15\x18<S\xc0...
1820788,1820789,300239305,SFO-2015MSMFC37034,NONFED,ST-NASF,ST/C&L,USMSMSS,Mississippi Forestry Commission,MSMFC,Mississippi Forestry Commission,...,B,31.142374,-89.406648,8.0,PRIVATE,MS,LAMAR,073,Lamar,b'\x00\x01\xad\x10\x00\x000%\xab\x85\x06ZV\xc0...
938731,938732,1066232,SWRA_TN_5236,NONFED,ST-TNTNS,ST/C&L,USTNTNS,Tennessee Division of Forestry,TNTNS6,TNS Unit 6,...,B,35.768300,-88.560000,14.0,MISSING/NOT SPECIFIED,TN,None,None,None,b'\x00\x01\xad\x10\x00\x00\xa0p=\n\xd7#V\xc00\...
245798,245799,250600,W-229284,FED,DOI-WFMI,NPS,USFLBCP,Big Cypress National Preserve,FLBCP,Big Cypress National Preserve,...,A,25.936310,-81.048760,3.0,NPS,FL,None,None,None,b'\x00\x01\xad\x10\x00\x00\x94VC\xe2\x1eCT\xc0...
1380979,1380980,19076669,SFO-GA-DAD-75-2/6/1992-1545,NONFED,ST-GAGAS,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,B,34.755900,-85.509700,8.0,PRIVATE,GA,Dade,083,Dade,b'\x00\x01\xad\x10\x00\x00X\xb1\xbf\xec\x9e`U\...


In [15]:
# Create a filtered dataframe from specific columns
fire_cols = ["FIRE_SIZE_CLASS", "LATITUDE", "LONGITUDE", "STATE","FIPS_CODE","FIPS_NAME"]
fire_tf= fire_df[fire_cols].copy()

# Rename the column headers
fire_tf = fire_tf.rename(columns={"FIRE_SIZE_CLASS": "class",
                                  "LATITUDE":"lat",
                                  "LONGITUDE":"long",
                                  "STATE":"state",
                                  "FIPS_CODE":"fcode",
                                  "FIPS_NAME":"fcounty"})

# Clean the data by dropping duplicates and setting the index
#fire_tf.drop_duplicates("id", inplace=True)
#fire_tf.set_index("id", inplace=True)

fire_tf.head()


,class,lat,long,state,fcode,fcounty
0,A,40.036944,-121.005833,CA,063,Plumas
1,A,38.933056,-120.404444,CA,061,Placer
2,A,38.984167,-120.735556,CA,017,El Dorado
3,A,38.559167,-119.913333,CA,003,Alpine
4,A,38.559167,-119.933056,CA,003,Alpine


In [16]:
connection_string = "root:less0001@localhost/fire_db"
engine = create_engine(f'mysql://{connection_string}')

ModuleNotFoundError: No module named 'MySQLdb'